In [2]:
import sqlite3
import pandas as pd

import os, sys
rootpath = os.path.join(os.getcwd(), '.')
sys.path.append(rootpath)
from src.utils import *
# reimport modules if they change
%load_ext autoreload
%autoreload 2

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f51fd1791e0>>
Traceback (most recent call last):
  File "/data.nst/johannes/envs/finite-observation/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
database = './simulations_nu0.2.db'

## Simulations

In [12]:
con = sqlite3.connect(database)
cur = con.cursor()
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS simulations (
            simulation_id integer PRIMARY KEY,
            N int NOT NULL,
            K int NOT NULL,
            lambda float NOT NULL,
            mu float NOT NULL,
            h float NOT NULL,
            window float NOT NULL,
            seed int NOT NULL,
            raw_file TEXT NOT NULL,
            dataset TEXT NOT NULL
            )
""")
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS parameters ON simulations (N,K,lambda,mu,h,window,seed);")
# can be dropped with cur.execute("DROP INDEX parameters;") if needs to be redesigned
con.commit()
con.close()

In [23]:
con = sqlite3.connect(database)
simulations = pd.read_sql_query("SELECT * FROM simulations", con)
con.close()
simulations

,simulation_id,N,K,lambda,mu,h,window,seed,raw_file,dataset
0,1,10000,100,0.000000,0.2,100.0,1.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/1.0
1,2,10000,100,0.000000,0.2,100.0,10.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/10.0
2,3,10000,100,0.000000,0.2,100.0,100.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/100.0
3,4,10000,100,0.000000,0.2,100.0,1000.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/1000.0
4,5,10000,100,0.000000,0.2,100.0,10000.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/10000.0
...,...,...,...,...,...,...,...,...,...,...
2970,2971,10000,100,0.999822,0.2,100.0,1.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/1.0
2971,2972,10000,100,0.999822,0.2,100.0,10.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/10.0
2972,2973,10000,100,0.999822,0.2,100.0,100.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/100.0
2973,2974,10000,100,0.999822,0.2,100.0,1000.0,1000,/scratch02.local/johannes/projects/sahel_finit...,samples/1000.0


In [25]:
import numpy as np
unique_seeds = np.sort(simulations.seed.unique())
print("simulation seeds: ", unique_seeds)

# check that all simulations have entered the database
unique_lambda = np.sort(simulations["lambda"].unique())
unique_h = np.sort(simulations["h"].unique())
for seed in unique_seeds:
    for l in unique_lambda:
        for (i,h) in enumerate(unique_h):
            windows = simulations[(simulations["seed"] == seed) & (simulations["lambda"] == l) & (simulations["h"] == h)]["window"].unique()
            if len(windows) != 5:
                print(f"error for seed = {seed}, lambda = {l}, h = {h} (index={i}) : windows = {windows}")

simulation seeds:  [1000]


In [1]:
# # fix missing simulations
# windows = simulations["window"].unique()
# hs = simulations["h"].unique()
# print("windows: ", windows)
# con = sqlite3.connect(database)
# cur = con.cursor()
# for l in unique_lambda:
#     h_local = simulations[simulations["lambda"] == l]["h"].unique()
#     h_missing = np.setdiff1d(hs, h_local)
#     print(f"l={l}, h_missing: ", h_missing)
#     # add missing simulations

#     for h in h_missing:
#         raw_file = f"/scratch02.local/johannes/projects/sahel_finite-observation-dynamic-range/nu=0.2/N=10000_K=100/seed=1000/1-lambda={1-l:.2e}/simulation_mu=0.20_h={h:.2e}.h5"
#         # check if file exists
#         if os.path.isfile(raw_file):
#             pass
#         else:
#             print(f"file {raw_file} does not exist")
#         for window in windows:
#             cur.execute(f"""
#             INSERT INTO simulations (N,K,lambda,mu,h,window,seed,raw_file,dataset)
#             VALUES (10000, 100, {l}, 0.2, {h}, {window}, 1000, "{raw_file}", "samples/{window}")
#             """)
#             con.commit()
# con.close()

## Beta Approximation

In [8]:
# create table that does not allow duplicates
con = sqlite3.connect(database)
cur = con.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS beta_approximations (
            simulation_id INTEGER NOT NULL UNIQUE,
            a float NOT NULL,
            b float NOT NULL,
            loc float NOT NULL,
            scale float NOT NULL,
            FOREIGN KEY (simulation_id) REFERENCES simulations (simulation_id)
            )
""")
con.commit()
con.close()

In [9]:
con = sqlite3.connect(database)
# show state of approximation simulations
beta_approximations = pd.read_sql_query("SELECT * FROM beta_approximations", con)
con.close()
beta_approximations

,simulation_id,a,b,loc,scale


In [10]:
# check if approximations are available for all simulation_ids
# for simulation_id in simulations.simulation_id:
#     if np.sum(beta_approximations.simulation_id==simulation_id) != 1:
#         print(f"error: not a unique approximation for simulation_id = {simulation_id} with parameters:\n{simulations[simulations.simulation_id == simulation_id].iloc[0]}")

## Beta Interpolation

In [11]:
con = sqlite3.connect(database)
cur = con.cursor()
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS beta_interpolations (
            N INTEGER NOT NULL,
            K INTEGER NOT NULL,
            mu FLOAT NOT NULL,
            seed INTEGER NOT NULL,
            filename TEXT NOT NULL
            )
""")
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS interpolation ON beta_interpolations (N,K,mu,seed);")
con.commit()
con.close()

In [12]:
con = sqlite3.connect(database)
# show state of approximation simulations
beta_interpolations = pd.read_sql_query("SELECT * FROM beta_interpolations", con)
beta_interpolations

,N,K,mu,seed,filename


## Results

In [3]:
# create table that does not allow duplicates
con = sqlite3.connect(database)
cur = con.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS results (
            N INTEGER NOT NULL,
            K INTEGER NOT NULL,
            mu FLOAT NOT NULL,
            seed INTEGER NOT NULL,
            window float NOT NULL,
            sigma float NOT NULL,
            epsilon float NOT NULL,
            filename TEXT NOT NULL
            )
""")
# do not allow duplicates
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS result ON results (N,K,mu,seed,window,sigma,epsilon);")
con.commit()
con.close()

In [21]:
con = sqlite3.connect(database)
results = pd.read_sql_query("SELECT * FROM results", con)
con.close()
results

,N,K,mu,seed,window,sigma,epsilon,filename
0,10000,100,0.2,1000,1.000000,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
1,10000,100,0.2,1000,10.000000,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
2,10000,100,0.2,1000,25.118864,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
3,10000,100,0.2,1000,3.981072,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
4,10000,100,0.2,1000,2.511886,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
5,10000,100,0.2,1000,6.309573,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
6,10000,100,0.2,1000,1.584893,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
7,10000,100,0.2,1000,15.848932,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
8,10000,100,0.2,1000,39.810717,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...
9,10000,100,0.2,1000,63.095734,0.01,0.1,./results/nu=0.2//sigma=0.01_epsilon=0.1/N=100...


In [22]:
import numpy as np
windows = 10**np.arange(0,4.2,0.2)
print("windows: ", windows, f" in total {len(windows)}")

con = sqlite3.connect(database)
cur = con.cursor()
results = pd.read_sql_query("SELECT * FROM results", con)
wins = results.window.unique()
print("windows in database: ", wins)
missing_windows = np.setdiff1d(windows, wins)
print("missing windows: ", missing_windows)

# check for results file
for window in missing_windows:

    filename = f"./results/nu=0.2//sigma=0.01_epsilon=0.1/N=10000_K=100_mu=0.2/results_simulation_seed=1000_window={window}.txt"
    if os.path.isfile(filename):
        pass
    else:
        print(f"file {filename} does not exist")

    # cur.execute(f"""
    # INSERT INTO results (N,K,mu,seed,window,sigma,epsilon,filename)
    # VALUES (10000, 100, 0.2, {seed}, {window}, 0.1, 0.1, "{filename}")
    # """)
    # con.commit()
con.close()



windows:  [1.00000000e+00 1.58489319e+00 2.51188643e+00 3.98107171e+00
 6.30957344e+00 1.00000000e+01 1.58489319e+01 2.51188643e+01
 3.98107171e+01 6.30957344e+01 1.00000000e+02 1.58489319e+02
 2.51188643e+02 3.98107171e+02 6.30957344e+02 1.00000000e+03
 1.58489319e+03 2.51188643e+03 3.98107171e+03 6.30957344e+03
 1.00000000e+04]  in total 21
windows in database:  [1.00000000e+00 1.00000000e+01 2.51188643e+01 3.98107171e+00
 2.51188643e+00 6.30957344e+00 1.58489319e+00 1.58489319e+01
 3.98107171e+01 6.30957344e+01 2.51188643e+02 6.30957344e+02
 3.98107171e+02 1.00000000e+03 2.51188643e+03 1.58489319e+03
 6.30957344e+03 3.98107171e+03 1.00000000e+04 1.58489319e+02
 1.00000000e+02]
missing windows:  [1.58489319e+00 3.98107171e+00 6.30957344e+00 1.58489319e+01
 2.51188643e+01 6.30957344e+01 2.51188643e+02 6.30957344e+02
 2.51188643e+03 6.30957344e+03]
file ./results/nu=0.2//sigma=0.01_epsilon=0.1/N=10000_K=100_mu=0.2/results_simulation_seed=1000_window=1.5848931924611134.txt does not exis

## Clean up 

In [16]:
from tqdm import tqdm
import h5py
import numpy as np

know_what_you_do = False
if know_what_you_do:
    missing=0
    database = "./simulations_nu0.2.db"
    con = sqlite3.connect(database)
    cur = con.cursor()
    simulations = pd.read_sql_query("SELECT * FROM simulations", con)
    # check that all simulations have correct windows and datasets
    windows=[1e0,1e1,1e2,1e3,1e4]

    unique_seeds = [1000]
    print("simulation seeds: ", unique_seeds)
    for seed in tqdm(unique_seeds):
        dbseed = simulations[(simulations.seed == seed)]
        # unique lambda values
        unique_lambdas = np.sort(dbseed["lambda"].unique())
        for lam in unique_lambdas:
            dblam = dbseed[(dbseed["lambda"] == lam)]
            # unique h values
            unique_hs = np.sort(dblam["h"].unique())
            for h in unique_hs:
                sim = dblam[(dblam["h"] == h)]
                file_sim = sim["raw_file"].values[0]
                with h5py.File(file_sim, 'r') as f:
                    for window in windows:
                        try:
                            dset_db = sim[sim["window"]==window]["dataset"].values[0]
                        except IndexError: 
                            missing+=1
                            print(f"window {window} not found for seed {seed}, lambda {lam}, h {h}")
                            # check if dataset exists in raw file
                            try:
                                dset_std = f'samples/{window:.1f}'                    
                                data = f[dset_std]
                                print(f"{dset_std} found: {data}")
                                print(f"add to database: {sim.N.values[0]},{sim.K.values[0]},{sim['lambda'].values[0]},{sim.mu.values[0]},{sim.h.values[0]},{window},{sim.seed.values[0]},'{file_sim}','{dset_std}")
                                cur.execute(f"INSERT INTO simulations (N,K,lambda,mu,h,window,seed,raw_file,dataset) VALUES ({sim.N.values[0]},{sim.K.values[0]},{sim['lambda'].values[0]},{sim.mu.values[0]},{sim.h.values[0]},{window},{sim.seed.values[0]},'{file_sim}','{dset_std}')")
                            except KeyError:
                                print(f"window {window} not found in {file_sim}")
    if not missing:
        print("all datasets are set correctly")
    else:
        print(f"{missing} datasets were missing windows")
    con.commit()
    con.close()

In [17]:
# test saving files with
if False:
    database = "./test.db"
    from src.simulation import *

    params={
        'N': int(1e2),
        'K': 10,
        'lambda': 0.0,
        'mu': 0.2,
        'h': 1,
        'seed': 1234,
    }
    result = simulation(params)
    save_simulation(result, path='./logs/', database=database, verbose=True)